In [62]:
import urllib, cStringIO, json, traceback
from PIL import Image
import i2v 
import sys

In [75]:
def getTags(key, illust2vec, img, file, url):
    labels = illust2vec.estimate_plausible_tags([img], threshold=0.07)
    temp = {}
    temp["id"]=key
    temp["labels"]=labels
    temp["url"]=url
    file.write(json.dumps(temp) + "\n")
    return labels

In [107]:
#initiate some vars
base_path="../../../testi2v/illustration2vec/data/"
json_data_path= base_path + 'anime-min.json'
scraped_data_path= base_path + 'test-2.json'
image_labels2_path = base_path + 'labels2-min.json'
error_path = base_path + 'labels-error-min.json'
image_labels_path = base_path +'labels-min.json'
anime_dict={} 
labels_dict = {}
anime_images_dict = {}

In [66]:
#load label model
illust2vec = i2v.make_i2v_with_chainer(
    "../../../testi2v/illustration2vec/illust2vec_tag_ver200.caffemodel", "../../../testi2v/illustration2vec/tag_list.json")

In [108]:
#load dict with images
with open(json_data_path,'r') as anime_dict_json:
	anime_dict = json.load(anime_dict_json)

In [109]:
#load new images
extra_images_array = []
with open(scraped_data_path,'r') as images_array_json:
	extra_images_array = json.load(images_array_json)

In [110]:
print extra_images_array

[{u'url': u'https://myanimelist.net/anime/890/Yuusha-Ou_GaoGaiGar/pics', u'image_urls': [[u'https://myanimelist.cdn-dena.com/images/anime/12/4306.jpg'], [u'https://myanimelist.cdn-dena.com/images/anime/12/9828.jpg'], [u'https://myanimelist.cdn-dena.com/images/anime/3/21182.jpg'], [u'https://myanimelist.cdn-dena.com/images/anime/12/21183.jpg'], [u'https://myanimelist.cdn-dena.com/images/anime/8/23242.jpg']]}, {u'url': u'https://myanimelist.net/anime/2151/Nils_no_Fushigi_na_Tabi/pics', u'image_urls': [[u'https://myanimelist.cdn-dena.com/images/anime/1/2296.jpg'], [u'https://myanimelist.cdn-dena.com/images/anime/11/9806.jpg'], [u'https://myanimelist.cdn-dena.com/images/anime/12/9807.jpg'], [u'https://myanimelist.cdn-dena.com/images/anime/6/9808.jpg'], [u'https://myanimelist.cdn-dena.com/images/anime/11/9809.jpg']]}, {u'url': u'https://myanimelist.net/anime/22507/Initial_D_Final_Stage/pics', u'image_urls': [[u'https://myanimelist.cdn-dena.com/images/anime/3/64833.jpg'], [u'https://myanimel

In [111]:
#
labels_dict = {}
try: 
    print "reading info from: "+image_labels2_path
    with open(image_labels2_path, "r") as f:
        for pic in f:
            pic_labels = json.loads(pic)
            if pic_labels["id"] in labels_dict:
                array = labels_dict[pic_labels["id"]] 
                array.append(pic_labels["url"])
                labels_dict[pic_labels["id"]] = array
            else:
                 labels_dict[pic_labels["id"]] = [pic_labels["url"]]
except IOError as e:
    print "Error reading in"
    print (e.errno)
    pass

reading info from: ../../../testi2v/illustration2vec/data/labels2-min.json


In [96]:
labels_dict

{u'100': [u'https://myanimelist.cdn-dena.com/images/anime/1/100.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/11/5244.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/3/11708.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/13/12764.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/3/12765.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/6/12766.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/10/20890.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/10/24920.jpg'],
 u'1000': [u'https://myanimelist.cdn-dena.com/images/anime/4/21236.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/8/21255.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/2/21259.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/3/36609.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/2/53199.jpg'],
 u'10080': [u'https://myanimelist.cdn-dena.com/images/anime/5/28649.jpg',
  u'https://myanimelist.cdn-dena.com/images/anime/5/29073.jpg',
  u'https:

In [112]:
def checkUri(key, url):
    if key  in labels_dict:
        if url in labels_dict[key]:
            return True
        else:
            return False
    else:
        return False

In [113]:
import re
#add id to dict that cotnains the image_urls
for page_images in extra_images_array:
    matchObj = re.search( r'anime/(\d+)', page_images['url'], re.M|re.I)
    page_images['id'] = matchObj.group(1)

In [130]:
def callAndWriteToFile(key, URL,illust2vec, f, error_path):
    try:
        #retrieve pic from url convert to data
        file = cStringIO.StringIO(urllib.urlopen(URL).read())
        #open image
        img = Image.open(file)
        getTags(key, illust2vec,img, f, URL)
        #break;
    except:
        e = sys.exc_info()[0]
        with open(error_path, "a") as eror_file:
            temp = {}
            temp["id"]=key
            temp["image"] = URL
            callstack = ''.join(traceback.format_exc())
            temp["tracestack"]= callstack
            eror_file.write(json.dumps(temp) + "\n")
        print e
        pass

In [100]:
anime_dict['32012']

{u'image': u'http://cdn.myanimelist.net/images/anime/10/76999.jpg',
 u'title': u'Utakata Overdrive'}

In [131]:
print "starting to write data to: "+ image_labels2_path
with open(image_labels2_path, "a") as f:
    for page in extra_images_array:
        key = str(page['id'])
        record = {}
        if key+'.0' in anime_dict:
            record = anime_dict[key+'.0']
        elif key in anime_dict:
            record = anime_dict[key]
        else:
            with open(error_path, "a") as eror_file:
                temp = {}
                temp["id"]=key
                temp["image"] = value["url"]
                temp["tracestack"]= 'cldnt parse key'
                eror_file.write(json.dumps(temp) + "\n")
                print key
                print record['image']
                continue
        if record['image'] is None:
            record['image'] = ''
        matchObj = re.search(r"anime/(.*)", record['image'], re.M|re.I)
        if matchObj is None:
            text_url = ''
        else:
            text_url = matchObj.group(1)
        for url_array in page['image_urls']:
            url = url_array[0]
            if str(url).find(text_url) == -1 and  not checkUri(key,url):
                callAndWriteToFile(key, url, illust2vec,  f, image_error_labels_path)
print "done"

starting to write data to: ../../../testi2v/illustration2vec/data/labels2-min.json
<type 'exceptions.IOError'>
done
